<a href="https://colab.research.google.com/github/nakamura196/ndl_ocr/blob/main/ndl_ocr_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NDLOCRの実行例 Ver.2：Google Driveを用いた画像の入力と結果の保存

Google Drive上のフォルダを指定して、当該フォルダに含まれる複数画像に対してOCR処理を実行し、指定したフォルダに認識結果を出力します。

**本ノートブック（Ver.2）では、PDFファイルやIIIFマニフェストファイルの入力をサポートしています。また、実行後に認識結果をマージしたテキストファイルをダウンロードすることができます。**

## 参考記事

https://zenn.dev/nakamura196/articles/b6712981af3384

## 参考にしたノートブック

@blue0620 さんが作成したノートブック

https://github.com/blue0620/NDLOCR-GoogleColabVersion/blob/main/NDLOCR_googlecolabversion.ipynb

## 更新内容
- 2023-06-15
  - Google Colabで実行できない不具合を解消しました。プログラムを改修してくださったNDLおよび@blue0620さんに感謝申し上げます。
- 2023-03-24
  - ファイル名に半角スペースを含む場合に発生する不具合を修正しました。
  - 初期セットアップに要する時間を改善しました。
- 2023-01-18
  - Google Colabの実行環境の更新に伴う本ノートブックの不具合を修正しました。（@blue0620さん、ありがとうございました。）
  - 初期セットアップに要する時間を4分ほどに短縮しました。
- 2022-07-05
  - ルビのテキスト化を行うか否かの設定を追加しました。
  - PDFを入力した際の不具合の修正しました。
  - 出力結果の提示方法を修正しました。
- 2022-05-02: 透明テキスト付きPDFの作成機能を追加しました。
  - 出力フォルダの直下にPDFファイルが作成されます。
  - 一部のオプションでは、実行後に自動的にPDFファイルがダウンロードされます。
  - 作成されるPDFファイルは一部不完全なため、引き続き改良予定です。
- 2022-05-03
  - @blue0620 さんが作成された読み順付与機能を追加しました。
  - https://twitter.com/blue0620/status/1521137895494787072

## 使用方法

- 「1.初期セットアップ」を実行してください。初回のみ実行が必要です。

- 「2. 設定」を変更してください。

## 1.初期セットアップ

４分ほど時間がかかります。初回のみ実行が必要です。

In [ ]:
#@title

# Google Drive関連
from google.colab import drive
drive.mount('/content/drive/')
from google.colab import files
from IPython.display import clear_output
!pip install kora
from kora.xattr import get_id

# 関連リポジトリ設定

%cd "/content/"

!rm -rf demo
!git clone https://github.com/nakamura196/demo

# 1. NDLOCRのリポジトリをcloneする(--recursiveを忘れずに！)
!rm -rf ndlocr_cli
!git clone --recursive https://github.com/ndl-lab/ndlocr_cli

# 2. 必要なパッケージをインストールする
PROJECT_DIR="/content/ndlocr_cli"

!pip install mmcv==2.0.0 -f https://download.openmmlab.com/mmcv/dist/cu118/torch2.0/index.html
!pip install mmdet==3.0.0
!pip install lmdb

# # 4. OCRに必要な学習済みモデルをダウンロードする
%cd $PROJECT_DIR
!wget https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/mojilist_NDL.txt -P ./src/text_recognition/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/ndlenfixed64-mj0-synth1.pth -P ./src/text_recognition/models
# wget https://lab.ndl.go.jp/dataset/ndlocr/ndl_layout/ndl_layout_config.py -P ./src/ndl_layout/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/ndl_layout/epoch_140_all_eql_bt.pth -P ./src/ndl_layout/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/separate_pages_ssd/weights.hdf5 -P ./src/separate_pages_ssd/ssd_tools

# 4.5. 拡張機能【読み順の自動ソート】を追加する
!rm ./cli/core/inference.py
!wget https://raw.githubusercontent.com/nakamura196/simple_reading_order/main/inference.py -P ./cli/core/
!wget https://lab.ndl.go.jp/dataset/ndlocr/appendix/simple_reading_order_model.joblib -P .

# 6. PDFを画像に変換するためのパッケージのインストール
!apt-get install poppler-utils
!pip install pdf2image

# transparent
!pip install python-bidi
!pip install reportlab

# 環境変数を追加する
import os
os.environ["PYTHONPATH"]=os.environ["PYTHONPATH"]+":"+f"{PROJECT_DIR}/src/text_recognition/deep-text-recognition-benchmark"

!mv /content/demo/*.py $PROJECT_DIR

# インポート
from task import Task

def message(ins):
  print("以下に出力しました。\nhttps://drive.google.com/drive/folders/{}".format(get_id(ins.output_dir)))

%cd $PROJECT_DIR

# 出力結果のクリア
clear_output()

## 2.設定

以下、入力方式によって適切なものを選んでください。

- 画像
  - [単一の画像ファイルのURLを指定する場合](#scrollTo=HugE-wRDxl5E)
  - [単一の画像ファイルをアップロードする場合](#scrollTo=Hx0mBi_jk9_s)
  - [複数の既にダウンロード済みの画像ファイルを対象にする場合](#scrollTo=OPICIT1-Z6_3)
- PDF
  - [単一のPDFファイルのURLを指定する場合](#scrollTo=jlitIIC5SkrR)
  - [単一のPDFファイルをアップロードする場合](#scrollTo=yK5kvjlY1T3K)
  - [単一の既にダウンロード済みのPDFファイルを対象にする場合](#scrollTo=4DFFxQECQtzJ)
  - [複数の既にダウンロード済みのPDFファイルを格納したフォルダを指定する場合](#scrollTo=VXRz_3tn60v5)
- IIIF
  - [IIIFマニフェストファイルのURLを指定する場合](#scrollTo=LhMvDLsXUZde)

## 画像

### 単一の画像ファイルのURLを指定する場合

- url: 画像ファイルのURL
- output_dir: 出力するフォルダへのパス
- ruby: ルビのテキスト化を行うか否か

入力サンプル：「校異源氏物語」（国立国会図書館所蔵）

In [ ]:
#@title 設定

url = "https://www.dl.ndl.go.jp/api/iiif/3437686/R0000006/full/full/0/default.jpg" #@param {type:"string"}
output_dir = "/content/drive/MyDrive/ndl_ocr/output/image_url" #@param {type:"string"}
process = "\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)" #@param ["\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u6587\u5B57\u8A8D\u8B58(OCR)"]
ruby = False #@param {type:"boolean"}

ins = Task.imgFromUrl(url, output_dir, process, ruby)
message(ins)

### 単一の画像ファイルをアップロードする場合

- output_dir: 出力するフォルダへのパス
- ruby: ルビのテキスト化を行うか否か

以下の設定の再生ボタンを押すと、ファイルのアップロードフォームが表示されます。

In [ ]:
#@title 設定

output_dir = "/content/drive/MyDrive/ndl_ocr/output/image_local" #@param {type:"string"}
process = "\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)" #@param ["\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u6587\u5B57\u8A8D\u8B58(OCR)"]
ruby = False #@param {type:"boolean"}

uploaded = files.upload()
input_file = next(iter(uploaded))

ins = Task.imgFromLocal(input_file, output_dir, process, ruby)
message(ins)

### 複数の既にダウンロード済みの画像ファイルを対象にする場合（Sigle input dir mode）

- input_dir: 入力するフォルダのパス
  - 指定したフォルダの下にimgフォルダを用意し、その中に画像を格納してください。
- output_dir: 出力するフォルダへのパス
- ruby: ルビのテキスト化を行うか否か

In [ ]:
#@title 設定

input_dir = "/content/drive/MyDrive/ndl_ocr/input" #@param {type:"string"}
output_dir = "/content/drive/MyDrive/ndl_ocr/output/image_single" #@param {type:"string"}
process = "\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)" #@param ["\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u6587\u5B57\u8A8D\u8B58(OCR)"]
ruby = False #@param {type:"boolean"}

ins = Task.imgFromLocalDir(input_dir, output_dir, process, ruby)
message(ins)

## PDF

### 単一のPDFファイルのURLを指定する場合

- url: PDFファイルのURL
- output_dir: 出力するフォルダへのパス
- ruby: ルビのテキスト化を行うか否か

入力サンプル：「東洋学芸雑誌」（人間文化研究機構国立国語研究所所蔵）

In [ ]:
#@title 設定

url = "https://dglb01.ninjal.ac.jp/ninjaldl/toyogakuge/001/PDF/tygz-001.pdf" #@param {type:"string"}
output_dir = "/content/drive/MyDrive/ndl_ocr/output/pdf_url" #@param {type:"string"}
process = "\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)" #@param ["\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u6587\u5B57\u8A8D\u8B58(OCR)"]
ruby = False #@param {type:"boolean"}

ins = Task.pdfFromUrl(url, output_dir, process, ruby)
message(ins)

### 単一のPDFファイルをアップロードする場合

- output_dir: 出力するフォルダへのパス
- ruby: ルビのテキスト化を行うか否か

以下の設定の再生ボタンを押すと、ファイルのアップロードフォームが表示されます。

In [ ]:
#@title 設定

output_dir = "/content/drive/MyDrive/ndl_ocr/output/image_local" #@param {type:"string"}
process = "\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)" #@param ["\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u6587\u5B57\u8A8D\u8B58(OCR)"]
ruby = False #@param {type:"boolean"}

uploaded = files.upload()
input_file = next(iter(uploaded))

ins = Task.pdfFromLocal(input_file, output_dir, process, ruby)
message(ins)

### 単一の既にダウンロード済みのPDFファイルを対象にする場合

- input_file: PDFファイルのパス
- output_dir: 出力するフォルダへのパス
- ruby: ルビのテキスト化を行うか否か

In [ ]:
#@title　設定

input_file = "/content/drive/MyDrive/ndl_ocr/input/pdfs/tygz-001.pdf" #@param {type:"string"}
output_dir = "/content/drive/MyDrive/ndl_ocr/output/pdf_local" #@param {type:"string"}
process = "\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)" #@param ["\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u6587\u5B57\u8A8D\u8B58(OCR)"]
ruby = False #@param {type:"boolean"}

ins = Task.pdfFromLocal(input_file, output_dir, process, ruby)
message(ins)

### 複数の既にダウンロード済みのPDFファイルを格納したフォルダを指定する場合

- input_dir: ダウンロード済みのPDFファイルを格納したフォルダ
- output_dir: 出力するフォルダへのパス
- ruby: ルビのテキスト化を行うか否か

In [ ]:
#@title　設定

input_dir = "/content/drive/MyDrive/ndl_ocr/input/pdfs" #@param {type:"string"}
output_dir = "/content/drive/MyDrive/ndl_ocr/output/pdfs_local" #@param {type:"string"}
process = "\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)" #@param ["\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u6587\u5B57\u8A8D\u8B58(OCR)"]
ruby = False #@param {type:"boolean"}

ins = Task.pdfFromLocalDir(input_dir, output_dir, process, ruby)
message(ins)

## IIIF

### IIIFマニフェストファイルのURLを指定する場合

- url: IIIFマニフェストファイルのURL
- output_dir: 出力するフォルダへのパス
- ruby: ルビのテキスト化を行うか否か
- process_size: 処理するcanvas数。-1を指定するとすべてのcanvasを処理対象とする。負荷軽減のため、初期値は5に設定しています。
- sleep_time: 画像ダウンロードの間隔（秒数）

入力サンプル：「校異源氏物語」（国立国会図書館所蔵）

In [ ]:
#@title 設定

url = "https://www.dl.ndl.go.jp/api/iiif/3437686/manifest.json" #@param {type:"string"}
output_dir = "/content/drive/MyDrive/ndl_ocr/output/iiif" #@param {type:"string"}
process = "\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)" #@param ["\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u6587\u5B57\u8A8D\u8B58(OCR)"]
ruby = False #@param {type:"boolean"}

process_size =   5#@param {type:"number"}
sleep_time =   1#@param {type:"number"}

ins = Task.iiif(url, output_dir, process, ruby, process_size, sleep_time)
message(ins)